In [1]:
import cv2
import numpy as np
from rplidar import RPLidar
import torch
import torchvision.transforms as T
from torchvision.models import resnet18
from IPython.display import clear_output, display
import ipywidgets as widgets
from IPython.display import HTML
from PIL import Image

In [2]:
!pip3 install flask


In [4]:

# Initialize webcam
cap = cv2.VideoCapture(0)

# Initialize LIDAR
lidar = RPLidar('/dev/ttyUSB0')
lidar.connect()
lidar.start_motor()

# Define detect_objects, process_lidar_data, and display_lidar_data_on_frame functions here
# ...
model = resnet18(pretrained=True)
model.eval()

def detect_objects(image, model):
    transform = T.Compose([
        T.Resize(256),
        T.CenterCrop(224),
        T.ToTensor(),
        T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    input_tensor = transform(cv2.cvtColor(image, cv2.COLOR_BGR2RGB)).unsqueeze(0)
    with torch.no_grad():
        output = model(input_tensor)
    return output


import numpy as np

def process_lidar_data(lidar_data, min_angle=-45, max_angle=45, min_distance=200):
    for scan in lidar_data:
        for point in scan:
            angle, distance, _ = point
            if min_angle <= angle <= max_angle and distance < min_distance:
                return True
    return False

def display_lidar_data_on_frame(frame, lidar_data, color=(0, 255, 0)):
    for scan in lidar_data:
        for point in scan:
            angle, distance, _ = point
            x = int(distance * np.cos(np.radians(angle)))
            y = int(distance * np.sin(np.radians(angle)))

            # Offset the points to display them at the center of the frame
            x += frame.shape[1] // 2
            y += frame.shape[0] // 2

            # Draw the point on the frame
            cv2.circle(frame, (x, y), 1, color, -1)
    return frame



# The main loop (as provided in the previous response)
try:
    while True:
        # Read image from webcam
        ret, frame = cap.read()
        if not ret:
            break

        # Perform object identification
        output = detect_objects(frame, model)

        # Get LIDAR data
        lidar_data = [item for item in lidar.iter_scans()]

        # Perform object avoidance
        if process_lidar_data(lidar_data):
            print("Obstacle detected! Take necessary action.")

        # Update the LIDAR frame
        lidar_frame = display_lidar_data_on_frame(frame.copy(), lidar_data)

except KeyboardInterrupt:
    print("Stopping...")
finally:
    cap.release()
    lidar.stop()
    lidar.disconnect()


TypeError: img should be PIL Image. Got <class 'numpy.ndarray'>

In [ ]:
from flask import Flask, Response, render_template
import threading

app = Flask(__name__)

# Function to generate a continuous stream of JPEG images
def generate_image_stream(frame_getter):
    while True:
        frame = frame_getter()
        _, jpeg = cv2.imencode('.jpg', frame)
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + jpeg.tobytes() + b'\r\n\r\n')

@app.route('/webcam')
def webcam_stream():
    return Response(generate_image_stream(lambda: frame),
                    content_type='multipart/x-mixed-replace; boundary=frame')

@app.route('/lidar')
def lidar_stream():
    return Response(generate_image_stream(lambda: lidar_frame),
                    content_type='multipart/x-mixed-replace; boundary=frame')

# Run the Flask web server in a separate thread
def run_flask_app():
    app.run(host='0.0.0.0', port=5000)

flask_thread = threading.Thread(target=run_flask_app)
flask_thread.start()

try:
    while True:
        # Read image from webcam
        ret, frame = cap.read()
        if not ret:
            break

        # Perform object identification
        output = detect_objects(frame, model)
        # You may want to process the output further to extract the detected objects and their labels

        # Get LIDAR data
        lidar_data = [item for item in lidar.iter_scans()]

        # Perform object avoidance
        if process_lidar_data(lidar_data):
            print("Obstacle detected! Take necessary action.")
            # Insert your control code here to take necessary action when an obstacle is detected

        # Update the LIDAR frame
        lidar_frame = display_lidar_data_on_frame(frame.copy(), lidar_data)

except KeyboardInterrupt:
    print("Stopping...")

finally:
    cap.release()
    if lidar._serial.is_open:
        lidar.stop()
        lidar.disconnect()


# Optionally, stop the Flask server
# flask_thread.join()


In [ ]:
from flask import Flask, Response, render_template
import threading

app = Flask(__name__)

# Function to generate a continuous stream of JPEG images
def generate_image_stream(frame_getter):
    while True:
        frame = frame_getter()
        _, jpeg = cv2.imencode('.jpg', frame)
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + jpeg.tobytes() + b'\r\n\r\n')

@app.route('/webcam')
def webcam_stream():
    return Response(generate_image_stream(lambda: frame),
                    content_type='multipart/x-mixed-replace; boundary=frame')

@app.route('/lidar')
def lidar_stream():
    return Response(generate_image_stream(lambda: lidar_frame),
                    content_type='multipart/x-mixed-replace; boundary=frame')

# Run the Flask web server in a separate thread
def run_flask_app():
    app.run(host='0.0.0.0', port=5000)

flask_thread = threading.Thread(target=run_flask_app)
flask_thread.start()
